In [1]:
import os
import sys

#sys.path.append("../../tools_service/src")
sys.path.append("../../common/src")
sys.path.append("../src")
os.chdir("../src")

In [2]:
PROJECT_ID = os.environ["PROJECT_ID"]

In [3]:
import config
from config import (VERTEX_LLM_TYPE_BISON_CHAT,
                    VERTEX_LLM_TYPE_BISON_TEXT,
                    VERTEX_AI_MODEL_GARDEN_LLAMA2_CHAT,
                    OPENAI_LLM_TYPE_GPT3_5,
                    OPENAI_LLM_TYPE_GPT4,
                    LLM_BACKEND_ROBOT_USERNAME,
                    LLM_BACKEND_ROBOT_PASSWORD)
from common.utils.token_handler import UserCredentials
config.TOOLS_SERVICE_BASE_URL = f"https://{PROJECT_ID}..cloudpssolutions.com/tools-service/api/v1"
config.auth_client = UserCredentials(LLM_BACKEND_ROBOT_USERNAME,
                              LLM_BACKEND_ROBOT_PASSWORD,
                              f"https:/{PROJECT_ID}.cloudpssolutions.com")

INFO: [config/config.py:54 - <module>()] Namespace File not found, setting job namespace as default
INFO: [config/config.py:87 - get_environ_flag()] ENABLE_GOOGLE_LLM = True
INFO: [config/config.py:87 - get_environ_flag()] ENABLE_GOOGLE_MODEL_GARDEN = True
INFO: [config/config.py:87 - get_environ_flag()] ENABLE_LLAMA2CPP_LLM = False
INFO: [config/config.py:87 - get_environ_flag()] ENABLE_OPENAI_LLM = True
INFO: [config/config.py:87 - get_environ_flag()] ENABLE_COHERE_LLM = True
INFO: [config/config.py:87 - get_environ_flag()] ENABLE_TRUSS_LLAMA2 = True
INFO: [config/config.py:248 - <module>()] Can't load llm_service_models.json: 404 Secret [projects/877759600672/secrets/llm_service_password_Llama2cpp] not found or has no versions.
INFO: [config/config.py:260 - <module>()] LLM types loaded ['OpenAI-GPT3.5', 'OpenAI-GPT4', 'Cohere', 'VertexAI-Text', 'VertexAI-Chat-V1', 'VertexAI-Chat']
INFO: [config/config.py:285 - <module>()] Embedding models loaded ['VertexAI-Embedding', 'OpenAI-Embedd

In [4]:
from config import OPENAI_API_KEY

ImportError: cannot import name 'OPENAI_API_KEY' from 'config' (/Users/sumeetvij/development/mira_v1/core-solution-services/components/llm_service/src/config/__init__.py)

In [6]:
from common.utils.secrets import get_secret

In [7]:
OPENAI_API_KEY = get_secret("openai-api-key")

In [8]:
from google.cloud import bigquery
from sqlalchemy import *
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import *
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor
from langchain.chat_models import ChatVertexAI
from langchain_experimental.sql import SQLDatabaseChain


service_account_file = "./data/gcp-mira-develop-gce-service-account.json" # Change to where your service account key file is located

dataset = "fqhc_medical_transactions"



sqlalchemy_url = f'bigquery://{PROJECT_ID}/{dataset}?credentials_path={service_account_file}'

In [9]:
db = SQLDatabase.from_uri(sqlalchemy_url)

/Users/sumeetvij/development/mira_v1/lib/python3.9/site-packages/google/cloud/bigquery/client.py:567: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  warnings.warn(


In [10]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [11]:
agent_executor = create_sql_agent(
                llm=llm,
                toolkit=toolkit,
                verbose=True,
                top_k=300
)

In [12]:
# First query
agent_executor.run("Which CDT codes are medicaid approved? ")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: CdtCode, ClaimTransaction, FqhcProvider
Thought: I should query the schema of the CdtCode table to see what columns I can use to answer the question.
Action: sql_db_schema
Action Input: CdtCode
Observation: 
CREATE TABLE `CdtCode` (
	`cdtCodeId` STRING, 
	`codeDescription` STRING, 
	`medicaidApproved` BOOL
)

/*
3 rows from CdtCode table:
cdtCodeId	codeDescription	medicaidApproved
D0145	oral evaluation for a patient under three years of age and counseling with primary caregiver	False
D0171	re-evaluation – post-operative office visit	False
D0190	screening of a patient	False
*/
Thought: I should query the CdtCode table for all CDT codes that are medicaid approved.
Action: sql_db_query
Action Input: SELECT cdtCodeId, codeDescription FROM CdtCode WHERE medicaidApproved = True LIMIT 300
Observation: [('D0120', 'periodic oral evaluation - established patient'), ('D0140', 'limited oral evaluation - 

'The CDT codes that are medicaid approved are D0120, D0140, D0150, D0160, D0170, D0180, D0210, D0220, D0230, D0240, D0250, D0270, D0320, D0330, D0340, D0350, D0370, and D0380.'

In [13]:
# Second query
agent_executor.run("""Provide the FQHC id and names that had a Claim Transaction that used a CDT code that was not Medicaid approved. 
Group the results by FQHC ID """)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: CdtCode, ClaimTransaction, FqhcProvider
Thought: I should query the schema of the CdtCode, ClaimTransaction, and FqhcProvider tables.
Action: sql_db_schema
Action Input: CdtCode, ClaimTransaction, FqhcProvider
Observation: 
CREATE TABLE `CdtCode` (
	`cdtCodeId` STRING, 
	`codeDescription` STRING, 
	`medicaidApproved` BOOL
)

/*
3 rows from CdtCode table:
cdtCodeId	codeDescription	medicaidApproved
D0145	oral evaluation for a patient under three years of age and counseling with primary caregiver	False
D0171	re-evaluation – post-operative office visit	False
D0190	screening of a patient	False
*/


CREATE TABLE `ClaimTransaction` (
	`claimTransactionId` INT64, 
	`fqhcProviderId` INT64, 
	`cdtCodeId` STRING, 
	`userId` INT64, 
	`userEmail` STRING, 
	`submitDateTime` TIMESTAMP
)

/*
3 rows from ClaimTransaction table:
claimTransactionId	fqhcProviderId	cdtCodeId	userId	userEmail	submitDateTime
85	380

"The FQHC ids and names that had a Claim Transaction that used a CDT code that was not Medicaid approved are: 5260779, '121ST STREET FAMILY HEALTH CENTER'; 275085, 'ACCESS SUPPORTS FOR LIVING INC'; 1928569, 'AHRC HEALTH CARE INC'; 2996601, 'AIDS CENTER QUEENS COUNTY INC'; 354114, 'ALICE HYDE MEDICAL CENTER'; 2994741, 'ALLEGANY REHABILITATION ASSOC'; 2977099, 'ALLEGANY REHABILITATION ASSOC INC'; 3801892, 'ANTHONY L JORDAN HEALTH CENTER'; 4073630, 'ANTHONY L JORDAN HEALTH CENTER'; 384969, 'ANTHONY L JORDAN HEALTH CTR'; 2994769, 'ASTOR HOME FOR CHILDREN'; 730483, 'BED STUY FAMILY HLTH CTR'; 1551555, 'AIDS CTR OF QUEENS COUNTY INC'; 740423, 'ALLEGANY REHABILITATION ASSOC';"

In [14]:
#use just SQLChain
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [15]:
db_chain.run("Which CDT codes are medicaid approved? ")



> Entering new SQLDatabaseChain chain...
Which CDT codes are medicaid approved? 
SQLQuery:SELECT cdtCodeId, codeDescription, medicaidApproved FROM CdtCode WHERE medicaidApproved = TRUE LIMIT 5;
SQLResult: [('D0120', 'periodic oral evaluation - established patient', True), ('D0140', 'limited oral evaluation - problem focused', True), ('D0150', 'comprehensive oral evaluation - new or established patient', True), ('D0160', 'detailed and extensive oral evaluation - problem focused, by report', True), ('D0170', 're-evaluation - limited, problem focused (established patient; not post-operative visit)', True)]
Answer:The CDT codes that are medicaid approved are D0120, D0140, D0150, D0160, and D0170.
> Finished chain.


'The CDT codes that are medicaid approved are D0120, D0140, D0150, D0160, and D0170.'

In [16]:
db_chain.run("""Provide the FQHC id and names that had a Claim Transaction that used a CDT code that was not Medicaid approved. 
Group the results by FQHC ID """)



> Entering new SQLDatabaseChain chain...
Provide the FQHC id and names that had a Claim Transaction that used a CDT code that was not Medicaid approved. 
Group the results by FQHC ID 
SQLQuery:SELECT fqhcProviderId, providerName FROM FqhcProvider 
INNER JOIN ClaimTransaction ON FqhcProvider.fqhcProviderId = ClaimTransaction.fqhcProviderId 
INNER JOIN CdtCode ON ClaimTransaction.cdtCodeId = CdtCode.cdtCodeId 
WHERE CdtCode.medicaidApproved = False 
GROUP BY fqhcProviderId, providerName 
LIMIT 5

DatabaseError: (google.cloud.bigquery.dbapi.exceptions.DatabaseError) 400 Column name fqhcProviderId is ambiguous at [1:8]

Location: US
Job ID: bd577638-492c-4f9b-817b-88fe243d938f

[SQL: SELECT fqhcProviderId, providerName FROM FqhcProvider 
INNER JOIN ClaimTransaction ON FqhcProvider.fqhcProviderId = ClaimTransaction.fqhcProviderId 
INNER JOIN CdtCode ON ClaimTransaction.cdtCodeId = CdtCode.cdtCodeId 
WHERE CdtCode.medicaidApproved = False 
GROUP BY fqhcProviderId, providerName 
LIMIT 5]
(Background on this error at: https://sqlalche.me/e/14/4xp6)

In [ ]:
#Use VertexAI

In [ ]:
LOCATION = 'us-central1'
MODEL_NAME= VERTEX_LLM_TYPE_BISON_TEXT

In [ ]:
llm = ChatVertexAI(project=PROJECT_ID, 
                   location=LOCATION, 
                   model_name= MODEL_NAME,
                   temperature=0.0, 
                   max_output_tokens=256)

In [ ]:
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [ ]:
agent_executor = create_sql_agent(
                llm=llm,
                toolkit=toolkit,
                verbose=True,
                top_k=300
)

In [ ]:
agent_executor.run("Which CDT codes are medicaid approved? ")
